## LSTM cell 
$$ i, j, f, o = Wx + Uh + b \\
   [i,j,f,o] = [W,U,1][x,h,b]^\top$$
  i = input_gate, j = new_input, f = forget_gate, o = output_gate


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
# %load unit_test.py
import tensorflow as tf
import numpy as np


from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope as vs

from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import nest

import sys, os
sys.path.append(os.path.abspath('../../'))

import tensornet
from tensorflow.python.ops.rnn_cell import *
from TensorBasicLSTMCell import TensorBasicLSTMCell


if __name__ == '__main__':
    np.random.seed(1)
    # the size of the hidden state for the lstm (notice the lstm uses 2x of this amount so actually lstm will have state of size 2)
    size = 12
    # 2 different sequences total
    batch_size= 2
    # the maximum steps for both sequences is 10
    n_steps = 10
    # each element of the sequence has dimension of 5
    seq_width = 2

    # the first input is to be stopped at 4 steps, the second at 6 steps
    e_stop = np.array([4,6])
    
    # factorize the inp
    inp_modes = {}
    out_modes = {}
    mat_ranks = {}
    # input weights
#     inp_modes['x'] = np.array([1, 2, 1, 1], dtype='int32') # product as seq_width
#     out_modes['x'] = np.array([1, 4, 3, 1], dtype='int32') # product as num_units (size)
    mat_ranks['x'] = 2
    #mat_ranks['x'] = np.array([1, 2, 2, 2, 1], dtype='int32')
    # hidden state weights
#     inp_modes['h'] = np.array([1, 4, 3, 1], dtype='int32') # seq_width
#     out_modes['h'] = np.array([1, 4, 3, 1], dtype='int32') # 4 * num_units
    mat_ranks['h'] = 2

    #mat_ranks['h'] = np.array([1, 2, 2, 2, 1], dtype='int32') 
    
    
    initializer = tf.random_uniform_initializer(-1,1)

    # the sequences, has n steps of maximum size
    seq_input = tf.placeholder(tf.float32, [n_steps, batch_size, seq_width], name="placeholder/seqs")
    # what timesteps we want to stop at, notice it's different for each batch hence dimension of [batch]
    early_stop = tf.placeholder(tf.int32, [batch_size], name="placeholder/stops" )

    # inputs for rnn needs to be a list, each item being a timestep.
    # we need to split our input into each timestep, and reshape it because split keeps dims by default
    # input = [n_steps, batch_size, seq_width]
    inputs = [tf.reshape(i, (batch_size, seq_width)) for i in tf.split(0, n_steps, seq_input)]
    
    """Shape checker"""
#     init = tf.initialize_all_variables()
#     sess = tf.InteractiveSession()
#     sess.run(init)
#     print(sess.run(tf.shape(inputs), feed_dict = {seq_input: np.ones((n_steps, batch_size, seq_width)) }))
#     sess.close()
    
    cell = TensorBasicLSTMCell(size, mat_ranks=mat_ranks, initializer=initializer)        
        
    initial_state = cell.zero_state(batch_size, tf.float32)

    # ========= This is the most important part ==========
    # output will be of length 4 and 6
    # the state is the final state at termination (stopped at step 4 and 6)
    outputs, state = tf.nn.rnn(cell, inputs, initial_state=initial_state, sequence_length=early_stop)

    # usual crap
    iop = tf.initialize_all_variables()
    session = tf.Session()
    session.run(iop)
    feed = {early_stop:e_stop, seq_input:np.random.rand(n_steps, batch_size, seq_width).astype('float32')}

    print("outputs, should be 2 things one of length 4 and other of 6")
    outs = session.run(outputs, feed_dict=feed)
    for xx in outs:
        print(xx)

    print("states, 2 things total both of size 2, which is the size of the hidden state")
    st = session.run(state, feed_dict=feed)
    print(st)



Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


outputs, should be 2 things one of length 4 and other of 6
[[ 0.00670183 -0.07692613  0.115038   -0.03025007 -0.06740323  0.03571117
   0.02862465  0.06593681 -0.00745523  0.01929194  0.07290111  0.01181453]
 [-0.00910865 -0.00448969  0.00168666 -0.04176504  0.00708137 -0.04368379
  -0.01599788  0.00225303  0.04981667  0.04072442  0.04412229 -0.04264941]]
[[ 0.00519152 -0.05586497  0.12005062 -0.01029339 -0.06654537  0.06891896
   0.04359735  0.06197734 -0.03885228 -0.00704982  0.07121877  0.04805806]
 [-0.00534405 -0.01214918  0.06271847 -0.03931961 -0.03459238 -0.0026452
  -0.00836922  0.034049    0.0207555   0.05222776  0.07461825 -0.02878173]]
[[ 0.01043024 -0.11395675  0.1858615  -0.02497832 -0.12027306  0.10930584
   0.07208047  0.08590036 -0.04051073 -0.00079288  0.10808595  0.06905206]
 [ 0.00333367 -0.07048583  0.16930935 -0.0476258  -0.10049284  0.04364385
   0.02125141  0.08519344 -0.00681046  0.04629445  0.12445677 -0.0076697 ]]
[[-0.00660907 -0.12951377  0.161722   -0.0812